Some comments

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import os
import numpy as np
repo_dir = '/datasets/hsc/repo/rerun/DM-13666/WIDE'
assert os.path.isdir(repo_dir)

import lsst.daf.persistence as daf_persistence
butler = daf_persistence.Butler(repo_dir)

In [5]:
coadd_skymap = butler.get('deepCoadd_skyMap')

In [6]:
dir(coadd_skymap)

['ConfigClass',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_decToRingNum',
 '_numTracts',
 '_raStart',
 '_raToTractNum',
 '_ringNums',
 '_ringSize',
 '_sha1',
 '_tractCache',
 '_tractInfo',
 '_tractInfoList',
 '_version',
 '_wcsFactory',
 'config',
 'findAllTracts',
 'findClosestTractPatchList',
 'findTract',
 'findTractPatchList',
 'generateTract',
 'getRingIndices',
 'getSha1',
 'logSkyMapInfo',
 'register',
 'updateSha1']

In [42]:
help(coadd_skymap.findTractPatchList)

Help on method findTractPatchList in module lsst.skymap.ringsSkyMap:

findTractPatchList(coordList) method of lsst.skymap.ringsSkyMap.RingsSkyMap instance
    Find tracts and patches that overlap a region.
    
    Parameters
    ----------
    coordList : `list` of `lsst.geom.SpherePoint`
        List of ICRS sky coordinates to search for.
    
    Returns
    -------
    reList : `list` of (`TractInfo`, `list` of `PatchInfo`)
        For tracts and patches that contain, or may contain, the specified
        region. The list will be empty if there is no overlap.
    
    Notes
    -----
    **warning:**
        This uses a naive algorithm that may find some tracts and patches
        that do not overlap the region (especially if the region is not a
        rectangle aligned along patch x, y).



In [10]:
import h5py
spatial_lookup_filename = '/project/danielsf/valid_hsc_visit_extent.h5'
assert os.path.isfile(spatial_lookup_filename)
spatial_lookup = h5py.File(spatial_lookup_filename, 'r')

In [33]:
spatial_lookup['visit'][11625]

6316

In [34]:
visit_num = 6316
all_visit = np.where(spatial_lookup['visit'][()]==visit_num)

In [37]:
ra_min = spatial_lookup['ra_min'][()][all_visit].min()
ra_max = spatial_lookup['ra_max'][()][all_visit].max()
dec_min = spatial_lookup['dec_min'][()][all_visit].min()
dec_max = spatial_lookup['dec_max'][()][all_visit].max()

In [40]:
all_coverage = np.where(np.logical_and(spatial_lookup['ra_center'][()]>ra_min,
                        np.logical_and(spatial_lookup['ra_center'][()]<ra_max,
                        np.logical_and(spatial_lookup['dec_center'][()]>dec_min,
                                       spatial_lookup['dec_center'][()]<dec_max))))

In [41]:
np.unique(spatial_lookup['filter'][()][all_coverage])

array([b'HSC-G', b'HSC-I', b'HSC-R', b'HSC-Y', b'HSC-Z'], dtype='|S5')

In [43]:
import lsst.geom as lsst_geom

In [50]:
pt_list = []
for ra in np.arange(ra_min, ra_max, 0.005):
    for dec in np.arange(dec_min, dec_max, 0.005):
        pt_list.append(lsst_geom.SpherePoint(ra, dec, lsst_geom.radians))

In [52]:
tract_patch_list = coadd_skymap.findTractPatchList(pt_list)

In [64]:
type(tract_patch_list[0][1][0])

lsst.skymap.patchInfo.PatchInfo

In [80]:
data_id = {'tract':tract_patch_list[0][0].getId(),
           'patch': str(tract_patch_list[0][1][0].getIndex()).replace('(','').replace(')','').replace(' ',''),
           'filter':'HSC-Y'}

In [70]:
butler.getKeys(datasetType='deepCoadd')

{'filter': str, 'tract': int, 'patch': str}

In [81]:
data_id

{'tract': 9451, 'patch': '0,5', 'filter': 'HSC-Y'}

In [85]:
data = butler.get('deepCoadd_forced_src', dataId=data_id)

In [93]:
#data.getSchema()